## Task - Cleaning and encoding green taxis dataset

- read the parquet file. tip: viewing the df can be very ugly if there are many features. you can choose the option vertical=TRUE to view each record as a column rather than a row.(`df.show(vertical=True)`)
- For columns 'ehail_fee' and 'congestion_surcharge' replace nulls with 0.
- For 'payment_type' replace the missing values the mode.
- Create a new column called `distance_km` that calculates the distance in km. It is currenlty in miles. 1 mile = 1.61 km.
- save the parquet file.

In [ ]:
## start the session
from pyspark.sql import SparkSession
# Initialize Spark session
spark = SparkSession.builder.appName("Lab8").getOrCreate()
# spark context to interact with the driver
sc = spark.sparkContext

In [ ]:
df=spark.read.parquet('/content/drive/MyDrive/Colab Notebooks/Datasets/green_tripdata_2015-07.parquet')

In [ ]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- lpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: integer (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: integer (nullable = true)



In [ ]:
df.summary().show()

+-------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+-------------------+-------------------+------------------+-------------------+---------+---------------------+------------------+------------------+------------------+--------------------+
|summary|           VendorID|store_and_fwd_flag|        RatecodeID|     PULocationID|     DOLocationID|   passenger_count|     trip_distance|       fare_amount|              extra|            mta_tax|        tip_amount|       tolls_amount|ehail_fee|improvement_surcharge|      total_amount|      payment_type|         trip_type|congestion_surcharge|
+-------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+-------------------+-------------------+------------------+-------------------+---------+---------------------+------------------+----------

In [ ]:
df.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2015-07-01 00:12:55|  2015-07-01 00:16:53|                 N|         1|         256|          37|              1|         0.88|        5.0|  0.5|    0.

In [ ]:
df.limit(5).show(vertical=True)

-RECORD 0------------------------------------
 VendorID              | 2                   
 lpep_pickup_datetime  | 2015-07-01 00:12:55 
 lpep_dropoff_datetime | 2015-07-01 00:16:53 
 store_and_fwd_flag    | N                   
 RatecodeID            | 1                   
 PULocationID          | 256                 
 DOLocationID          | 37                  
 passenger_count       | 1                   
 trip_distance         | 0.88                
 fare_amount           | 5.0                 
 extra                 | 0.5                 
 mta_tax               | 0.5                 
 tip_amount            | 1.58                
 tolls_amount          | 0.0                 
 ehail_fee             | NULL                
 improvement_surcharge | 0.3                 
 total_amount          | 7.88                
 payment_type          | 1                   
 trip_type             | 1.0                 
 congestion_surcharge  | NULL                
-RECORD 1-------------------------

In [ ]:
df.filter(df.ehail_fee.isNull()).count()

1541671

In [ ]:
df.filter(df.congestion_surcharge.isNull()).count()

1541671

In [ ]:
df=df.fillna(value=0,subset=['ehail_fee','congestion_surcharge'])

In [ ]:
df.filter(df.ehail_fee.isNull()).count(),df.filter(df.congestion_surcharge.isNull()).count()

(0, 0)

In [ ]:
df.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2015-07-01 00:12:55|  2015-07-01 00:16:53|                 N|         1|         256|          37|              1|         0.88|        5.0|  0.5|    0.

# replace nulls

In [ ]:
df.filter(df.payment_type.isNull()).count()
df.select('payment_type').distinct().show()

+------------+
|payment_type|
+------------+
|           5|
|           1|
|           3|
|           2|
|           4|
+------------+



In [ ]:
from pyspark.sql.functions import first
mode_payment_type = df.select(first("payment_type", ignorenulls=True)).first()[0]
mode_payment_type

1

In [ ]:
mode_payment_type = df.groupBy('payment_type').count().orderBy(col('count').desc()).limit(1)
mode_payment_type=mode_payment_type.select('payment_type').collect()[0][0]

In [ ]:
df=df.fillna(value=mode_payment_type,subset=['payment_type'])
df.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2015-07-01 00:12:55|  2015-07-01 00:16:53|                 N|         1|         256|          37|              1|         0.88|        5.0|  0.5|    0.

# new column called distance_km

In [ ]:
df=df.withColumn('distance_km',df.trip_distance*1.61)
df.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|       distance_km|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+------------------+
|       2| 2015-07-01 00:12:55|  2015-07-01 00:16:53|                 N|         1|         256|          

# **Saving**

In [ ]:
df.write.parquet("/content/drive/MyDrive/Colab Notebooks/Datasets/green_tripdata_2015-07_cleaned.parquet")

In [ ]:
## stop the driver
sc.stop()